In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import iminuit

from likelihoodtools import LikelihoodGridScanner, AdaptiveLikelihoodScanner, GridFitter, FunctionFitter, GaussianLikelihoodModel, ZeroPhaseGLikelihoodModel
from likelihoodtools import make_1d_llh_plot, make_2d_llh_plot
from likelihoodtools import add_noise

#in this demo sin_model_data is the function f that generates the noise free data
#due to the parallelization it has to be defined outside the notebook. It only works if it is defined in the top level of a 
#module. For reasons. (That means lambdas or local functions in functions don't work)
from sinemodel import sin_model_data, ChirpModel, SinModel

from tsp.data import noise_var, kb

In [ ]:
def make_experiment(theta, likelihood_model):
    
    data = likelihood_model.f(*theta)
    data = add_noise(data, likelihood_model)
    
    return data

In [ ]:
def tau_SNR(sigma, A, dt):
    return dt*sigma**2/A**2

def omega_CRLB_Joe(sigma, A, dt, tau):
    n = tau//dt # np.int64(tau/dt)
    tau_snr = tau_SNR(sigma, A, dt)
    return 96*tau_snr/tau**3

def omega_CRLB_90(sigma, A, dt, tau):
    n = tau//dt # np.int64(tau/dt)
    tau_snr = tau_SNR(sigma, A, dt)
    return 90*tau_snr/tau**3

def omega_CRLB_chirp_asymmetric(sigma, A, dt, tau):
    n = tau//dt # np.int64(tau/dt)
    tau_snr = tau_SNR(sigma, A, dt)
    return 6*tau_snr*(8*n-11)*(2*n-1)/(n*(n**4-5*n**2 + 4)*dt**3)

def omega_CRLB_symmetric(sigma, A, dt, tau):
    n = tau//dt
    tau_snr = tau_SNR(sigma, A, dt)
    return 6*tau_snr/((n**3-n)*dt**3)

def omega_CRLB_sine_asymmetric(sigma, A, dt, tau):
    n = tau//dt
    tau_snr = tau_SNR(sigma, A, dt)
    return 3*tau_snr/((2*n**3+n-3*n**2)*dt**3)

## Chirpmodel

In [ ]:
sr = 200.e6
dt = 1/sr
tau = 50.e-6

chirp_model = ChirpModel(sr, tau)

w0 = 20.e6*2*np.pi
alpha = 300.e6*2*np.pi
phi0 = 0.
t0 = 700.e-6 # 0.

P = .1e-17
R = 50.
T = 5.
sigma = np.sqrt(noise_var(T, sr))
A = np.sqrt(P*R)

In [ ]:
sigma_levels = [1]
fitter = GridFitter(sigma_levels=sigma_levels)

ax_names = ['A','w','alpha','phi0', 't0']
truth = [A, w0, alpha, phi0, t0]
truth_sine = [A, w0, phi0, t0]

In [ ]:
tau = 300.e-6

chirp_model = ChirpModel(sr, tau)

n = [0, 101, 0, 0, 0]
n_eval = [0, 2001, 0, 0, 0]

dw = 2*np.pi*5.e3

delta = [0, dw, 0, np.pi, 0]

model = GaussianLikelihoodModel(chirp_model, sigma, complex_data=True)
llh_scanner_1d = LikelihoodGridScanner(truth, delta, n_eval, ax_names, model, n, max_workers=1)

tau_vals = np.linspace(1.0e-6, 500.e-6, 14)

In [ ]:
def get_minuit_result(model_f, truth, y=None, minos=False, zero_phase=True, sigma=sigma):

    if zero_phase:
        model = ZeroPhaseGLikelihoodModel(model_f, sigma)
    else:
        model = GaussianLikelihoodModel(model_f, sigma, complex_data=True)

    if y is None:
        cost_f = lambda theta: -model.asimov_log_likelihood_function(truth)(theta)

    else:
        cost_f = lambda theta: -model.log_likelihood_function(y)(theta)

    cost_f.errordef = iminuit.Minuit.LIKELIHOOD

    guess = np.array(truth)
    
    minimizer = iminuit.Minuit(cost_f, guess)

#    minimizer.fixed['x0'] = True
#    minimizer.fixed['x1'] = True
#    minimizer.fixed['x2'] = True
#    minimizer.fixed['x3'] = True

   # minimizer.limits['x0'] = (1e-19, None)
   # minimizer.limits['x1'] = (1e-19, None)
   # minimizer.limits['x2'] = (1e-19, None)
   # minimizer.limits['x3'] = (-np.pi, np.pi)
   # minimizer.limits['x4'] = (1e-19, None)
    
    #minimizer.tol = 10.e4

    minimizer.migrad()

    minimizer.hesse()

    if minimizer.valid and minos:
        minimizer.minos()

    return minimizer

In [ ]:
model_f_unsmooth = ChirpModel(sr,tau_vals[5])
model_f= ChirpModel(sr,tau_vals[5], smoothing_factor=3000000.)

model = GaussianLikelihoodModel(model_f, sigma, complex_data=True)
model_zerophase = ZeroPhaseGLikelihoodModel(model_f, sigma)

model_unsmooth = GaussianLikelihoodModel(model_f_unsmooth, sigma, complex_data=True)
model_zerophase_unsmooth = ZeroPhaseGLikelihoodModel(model_f_unsmooth, sigma)

cost_f = model.asimov_log_likelihood_function(truth)
cost_f_zero = model_zerophase.asimov_log_likelihood_function(truth)
cost_f_unsmooth = model_unsmooth.asimov_log_likelihood_function(truth)
cost_f_zero_unsmooth = model_zerophase_unsmooth.asimov_log_likelihood_function(truth)

In [ ]:
data, window = model_f_unsmooth(*truth, return_window=True)

plt.plot()
plt.plot(data.real)
#plt.plot(window*np.max(data.real))
#plt.xlim((1800,3200))
plt.show()

data, window = model_f(*truth, return_window=True)

plt.plot(data.real, label='Windowed Model')
plt.plot(window*np.max(data.real), label='Windowing function')
plt.xlabel('sample #')
plt.ylabel('A [a.u.]')
plt.legend(loc='upper right')
plt.savefig('windowed_model.png', dpi=400)
plt.show()


In [ ]:
t_pm = 10.2e-6
t_off = np.linspace(truth[-1]-t_pm, truth[-1]+t_pm, 5001)

llh_vals = []
llh_vals_mod = []

llh_vals_unsmooth = []
llh_vals_mod_unsmooth = []

for t_ in t_off:
    theta = np.array(truth)
    theta[-1] = t_
    
    llh_vals.append(cost_f(theta))
    llh_vals_mod.append(cost_f_zero(theta))

    llh_vals_unsmooth.append(cost_f_unsmooth(theta))
    llh_vals_mod_unsmooth.append(cost_f_zero_unsmooth(theta))


In [ ]:
plt.plot(t_off, llh_vals, label='orig')
plt.plot(t_off, llh_vals_mod, label='mod')
plt.xlabel('t0 [s]')
plt.ylabel('llh')
plt.legend()
#plt.xlim((0.98e-5,1.02e-5))
#plt.ylim((-0.003,0.002))
plt.savefig('llh_llh_mod.png', dpi=400)
plt.show()

ylim = (-0.1,0.01)
plt.plot(t_off, llh_vals_unsmooth, label='orig')
plt.plot(t_off, llh_vals_mod_unsmooth, label='mod')
plt.xlabel('t0 [s]')
plt.ylabel('llh')
plt.legend()
#plt.xlim((0.98e-5,1.02e-5))
plt.ylim(ylim)
plt.savefig('llh_llh_mod_zoom_not_smooth.png', dpi=400)
plt.show()

plt.plot(t_off, llh_vals, label='orig')
plt.plot(t_off, llh_vals_mod, label='mod')
plt.xlabel('t0 [s]')
plt.ylabel('llh')
plt.legend()
#plt.xlim((0.98e-5,1.02e-5))
plt.ylim(ylim)
plt.savefig('llh_llh_mod_zoom_smooth.png', dpi=400)
plt.show()

In [ ]:
smoothing_factor = np.linspace(20000., 6000000., 30)
#smoothing_factor = np.logspace(4, 7, 20)

for i, s in enumerate([smoothing_factor[0], smoothing_factor[-1]]):
    print(s)
    model_f= ChirpModel(sr,tau_vals[-1], smoothing_factor=s)

    data, window = model_f(*truth, return_window=True)

    plt.plot()
    plt.plot(data.real, label='Windowed Model')
    plt.plot(window*np.max(data.real), label='Windowing Function')
    plt.xlabel('sample #')
    plt.ylabel('A [a.u.]')
    plt.legend(loc='upper right')
    plt.savefig(f'ts_window_{i}.png', dpi=400)
    plt.show()

In [ ]:
sigma2 = np.sqrt(noise_var(10., sr))

In [ ]:
errors = []
merrors_l = []
merrors_u = []
minimizers = []

errors2 = []
merrors_l2 = []
merrors_u2 = []
minimizers2 = []

for s in smoothing_factor:
    print(s)
    minimizer = get_minuit_result(ChirpModel(sr,tau_vals[5], smoothing_factor=s), truth, minos=True)
    minimizer2 = get_minuit_result(ChirpModel(sr,tau_vals[5], smoothing_factor=s), truth, minos=True, sigma=sigma2)
    errors.append(minimizer.errors['x4'])
    merrors_l.append(-minimizer.merrors['x4'].lower)
    merrors_u.append(minimizer.merrors['x4'].upper)
    minimizers.append(minimizer)
    
    errors2.append(minimizer2.errors['x4'])
    merrors_l2.append(-minimizer2.merrors['x4'].lower)
    merrors_u2.append(minimizer2.merrors['x4'].upper)
    minimizers2.append(minimizer2)

In [ ]:
minimizer_s1 = get_minuit_result(ChirpModel(sr,tau_vals[5], smoothing_factor=None), truth, minos=True)
minimizer_s2 = get_minuit_result(ChirpModel(sr,tau_vals[5], smoothing_factor=None), truth, minos=True, sigma=sigma2)

In [ ]:
first = 1

plt.plot(smoothing_factor[first:], errors[first:], label='Hesse error T=5K')
plt.plot(smoothing_factor[first:], errors2[first:], label='Hesse error T=10K')
plt.plot(smoothing_factor[first:], merrors_l[first:], label='Minos error T=5K')
plt.plot(smoothing_factor[first:], merrors_l2[first:], label='Minos error T=10K')
#plt.plot(smoothing_factor, merrors_u, label='Minos upper')
plt.axhline(minimizer_s1.errors['x4'], ls='--', color='red', label='Hesse Rec window T=5K')
plt.axhline(minimizer_s2.errors['x4'], ls='--', color='black', label='Hesse Rec window T=10K')
#plt.axhline(minimizer.merrors['x4'].upper)
plt.yscale('log')
#plt.xscale('log')
plt.ylabel(r'$\Delta$ t0 [s]')
plt.xlabel('smoothing parameter')
plt.legend()
plt.savefig('smoothing_t0_error.png', dpi=400)
plt.show()

In [ ]:
first = 1

plt.plot(smoothing_factor[first:], np.array(errors[first:])/np.array(merrors_l[first:]), label='Hesse error/Minos error T=5K')
plt.plot(smoothing_factor[first:], np.array(errors2[first:])/np.array(merrors_l2[first:]), label='Hesse error/Minos error T=10K')
#plt.plot(smoothing_factor[first:], merrors_l[first:], label='Minos error T=5K')
#plt.plot(smoothing_factor[first:], merrors_l2[first:], label='Minos error T=10K')

#plt.yscale('log')
#plt.xscale('log')
plt.ylabel(r'ratio')
plt.xlabel('smoothing parameter')
plt.legend()
plt.savefig('hesse_minos_ratio.png', dpi=400)
plt.show()

In [ ]:
def get_result(tau):
    chirp_model = ChirpModel(sr, tau)
    n = [0, 51, 0, 0, 0]
    n_eval = [0, 201, 0, 0, 0]

    dw = 2*np.pi*50.e3

    if tau < 20.e-6:
        dw = 2*np.pi*500.e3

    if tau > 80.e-6:
        dw = 2*np.pi*10.e3

    if tau > 200.e-6:
        dw = 2*np.pi*1.e3

    dw = np.sqrt(omega_CRLB_sine_asymmetric(sigma, A, 1/sr, tau))*5

    delta = [0, dw, 0, np.pi, 0]

    model = GaussianLikelihoodModel(chirp_model, sigma, complex_data=True)
    llh_scanner_1d = LikelihoodGridScanner(truth, delta, n_eval, ax_names, model, n, max_workers=1)
    llh_scan_1d = llh_scanner_1d.get_asimov_scan()

    fit_result_1d = fitter.get_best_fit_with_errors(llh_scan_1d,parameters_of_interest=1)

    return fit_result_1d

In [ ]:
fit_result_ = get_result(1.0e-6)

In [ ]:
make_1d_llh_plot(fit_result_)

In [ ]:
fit_results = []

for tau in tau_vals:
    fit_results.append(get_result(tau))

In [ ]:
dw = []

for fit_result in fit_results:
    make_1d_llh_plot(fit_result)
    dw.append(fit_result.errors[0][1][0])

In [ ]:
def scan_tau_vals_minuit(tau_vals, model, truth, y=None):

    error_hesse = []
    error_minos = []
    final_tau_vals = []
    minimizer_array = []

    for i, tau in enumerate(tau_vals):
        print(i, tau)

        minimizer = get_minuit_result(model(sr, tau), truth, y)
        minimizer_array.append(minimizer)
        if minimizer.valid:
            error_minos.append(minimizer.merrors[1].upper)
            error_hesse.append(minimizer.errors[1])
            final_tau_vals.append(tau)

    return error_hesse, error_minos, final_tau_vals, minimizer_array

In [ ]:
error_hesse_chirp, error_minos_chirp, tau_vals_chirp, minimizer_array_chirp = scan_tau_vals_minuit(tau_vals, ChirpModel, truth)

In [ ]:
error_hesse_sin, error_minos_sin, tau_vals_sin, minimizer_array_sin = scan_tau_vals_minuit(tau_vals, SinModel, truth_sine)

In [ ]:
cut_ind = 0
tau_cut = np.array(tau_vals_chirp[cut_ind:])
df_hesse = np.array(error_hesse_chirp[cut_ind:])/(2*np.pi)
df_minos = np.array(error_minos_chirp[cut_ind:])/(2*np.pi)
df_llhtools = np.array(dw[cut_ind:])/(2*np.pi)

tau_cut_sin = np.array(tau_vals_sin[cut_ind:])
df_hesse_sin = np.array(error_hesse_sin[cut_ind:])/(2*np.pi)
df_minos_sin = np.array(error_minos_sin[cut_ind:])/(2*np.pi)

CRLB_vals_asymmetric = np.sqrt(omega_CRLB_chirp_asymmetric(sigma, A, 1/sr, tau_cut))/(2*np.pi)
CRLB_vals_symmetric = np.sqrt(omega_CRLB_symmetric(sigma, A, 1/sr, tau_cut))/(2*np.pi)
CRLB_vals_sine = np.sqrt(omega_CRLB_sine_asymmetric(sigma, A, 1/sr, tau_cut))/(2*np.pi)
CRLB_vals_P8 = np.sqrt(omega_CRLB_Joe(sigma, A, 1/sr, tau_cut))/(2*np.pi) 
CRLB_vals_P8_90 = np.sqrt(omega_CRLB_90(sigma, A, 1/sr, tau_cut))/(2*np.pi) 

plt.plot(tau_cut/1.e-6, df_minos/1e3, marker='v', label='Minuit - Minos')
plt.plot(tau_cut/1.e-6, df_hesse/1e3, marker='*', label='Minuit - Hesse', markersize=10)
#plt.plot(tau_cut/1.e-6, df_minos_sin/1e3, marker='*', label='LLH sinus', markersize=10)
plt.plot(tau_cut/1.e-6, CRLB_vals_asymmetric/1e3, marker='^', label='CRLB 4 param')
#plt.plot(tau_cut/1.e-6, CRLB_vals_symmetric/1e3, marker='x', label='exact CRLB symmetric/Sinus')
#plt.plot(tau_cut/1.e-6, CRLB_vals_P8/1e3, marker='x', label='CRLB Joe')
plt.plot(tau_cut/1.e-6, CRLB_vals_sine/1e3, marker='*', label='CRLB 1 param', markersize=10)
plt.plot(tau_cut/1.e-6, df_llhtools/1e3, marker='v', label='LLH scan')

plt.xlabel(r'$\tau$ [us]')
plt.ylabel(r'$\Delta f$ [kHz]')
plt.yscale('log')
plt.legend()
plt.savefig('CRLB_comparison_chirp.png', dpi=600)
plt.show()

plt.plot(tau_cut/1.e-6, df_hesse/CRLB_vals_asymmetric, marker='*', label='Minuit - Hesse / CRLB 4 param')
plt.plot(tau_cut/1.e-6, df_minos/CRLB_vals_asymmetric, marker='*', label='Minuit - Minos / CRLB 4 param')
#plt.plot(tau_cut/1.e-6, CRLB_vals_symmetric/CRLB_vals_asymmetric, marker='*', label='CRLB symmetric')
#plt.plot(tau_cut/1.e-6, CRLB_vals_P8/CRLB_vals_asymmetric, marker='*', label='CRLB Joe / CRLB exact chirp')
#plt.plot(tau_cut/1.e-6, df_minos_sin/CRLB_vals_symmetric, marker='*', label='LLH sine / CRLB exact sine')
plt.plot(tau_cut[3:]/1.e-6, df_llhtools[3:]/CRLB_vals_sine[3:], marker='*', label='LLH scan / CRLB 1 param')
#plt.plot(tau_cut/1.e-6, CRLB_vals_P8_90/CRLB_vals_asymmetric, marker='*', label='CRLB 90')
plt.xlabel(r'$\tau$ [us]')
plt.ylabel('ratio')
#plt.yscale('log')
plt.legend()
plt.savefig('CRLB_comparison_chirp_ratio.png', dpi=600)
plt.show()

In [ ]:
slopes = np.linspace(0., alpha/10, 10)
slopes = np.concatenate((np.array([0.]),np.logspace(6, 10, 10)))

tau = tau_vals[int(len(tau_vals)/2)]
error_minos_slopes = []
error_minos_sine_slopes = []

for slope in slopes:
    truth_mod = np.array(truth)
    truth_mod[2] = slope
    _, error_minos, _, _ = scan_tau_vals_minuit([tau], ChirpModel, truth_mod)

    model = GaussianLikelihoodModel(ChirpModel(sr, tau), sigma, complex_data=True)

    y = model.f(*truth_mod)
    _, zero_slope_error, _, _ = scan_tau_vals_minuit([tau], SinModel, truth_sine, y)

    error_minos_slopes.append(error_minos[0])
    error_minos_sine_slopes.append(zero_slope_error[0])

In [ ]:
CRLB_asymmetric = np.sqrt(omega_CRLB_chirp_asymmetric(sigma, A, 1/sr, tau))/(2*np.pi*1e3)
CRLB_symmetric = np.sqrt(omega_CRLB_symmetric(sigma, A, 1/sr, tau))/(2*np.pi*1e3)
eta = slopes*tau/(2*w0)

plt.plot(eta, np.array(error_minos_slopes)/(2*np.pi*1e3) , marker='x', label='LLH chirp')
plt.plot(eta, np.array(error_minos_sine_slopes)/(2*np.pi*1e3) , marker='x', label='LLH sinus')
plt.axhline(CRLB_asymmetric, label='CRLB chirp', ls='--', color='red')
plt.axhline(CRLB_symmetric, label='CRLB sinus', ls='--', color='black')
plt.xlabel(r'$\eta$')
plt.ylabel(r'$\Delta f$ [kHz]')
plt.xscale('symlog',linthresh=1e-6)
#plt.ylim((0, 35))
plt.xlim((-1e-7, 2.e-2))
plt.legend()
plt.savefig('df_slope.png', dpi=400)
plt.show()